In [1]:
# import libraries
import numpy as np
import pandas as pd
import pymysql

In [2]:
df_item = pd.read_xml("C:/Nachiko/Artificial Intelligence/Black Elephant/Weekend_January_2025/Data Engineering Bootcamp/df_item.xml")
df_outlet = pd.read_xml("C:/Nachiko/Artificial Intelligence/Black Elephant/Weekend_January_2025/Data Engineering Bootcamp/df_outlet.xml")
df_sales = pd.read_xml("C:/Nachiko/Artificial Intelligence/Black Elephant/Weekend_January_2025/Data Engineering Bootcamp/df_sales.xml")

In [3]:
df_item.head()

,ID,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP
0,1,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092
1,2,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692
2,3,FDN15,17.50,Low Fat,0.016760,Meat,141.6180
3,4,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950
4,5,NCD19,8.93,Low Fat,0.000000,Household,53.8614


In [4]:
df_outlet.head()

,ID,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,1,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,2,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,3,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,4,OUT010,1998,Medium,Tier 3,Grocery Store
4,5,OUT013,1987,High,Tier 3,Supermarket Type1


In [5]:
df_sales.head()

,ID,Item_Outlet_Sales
0,1,3735.1380
1,2,443.4228
2,3,2097.2700
3,4,732.3800
4,5,994.7052


In [6]:
# Connection to my sql database
conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = 'Mysql@2025',
    autocommit = True
)
cursor =conn.cursor()
print(cursor)

In [7]:
cursor.execute("CREATE DATABASE IF NOT EXISTS BigMart;")
cursor.execute("USE BigMart;")

0

In [8]:
# Create tables
def create_table_from_df(df,table_name):
    cols =[]
    for col in df.columns:
        dtype = df[col].dtypes
        if "int" in str(dtype):
            sql_dtype = "INT"
        if "float" in str(dtype):
            sql_dtype = "FLOAT"
        else:
            sql_dtype = "VARCHAR(255)"
        cols.append(f"{col} {sql_dtype}")
    columns_sql = ','.join(cols)
    create_sql =f"CREATE TABLE IF NOT EXISTS {table_name} ({columns_sql})"
    cursor.execute(create_sql)

In [9]:
create_table_from_df(df_item, "item_info")
create_table_from_df(df_outlet, "outlet_info")
create_table_from_df(df_sales, "sales_info")

In [10]:
# insert data
def insert_data_from_df(df, table_name):
    df = df.where(pd.notnull(df),None)
    cols =",".join([f"{col}" for col in df.columns])
    placeholders = ",".join(["%s"]* len(df.columns))
    insert_sql = f"INSERT INTO {table_name} ({cols}) VALUES ({placeholders})"
    data = [tuple(row) for row in df.itertuples(index = False, name = None)]
    cursor.executemany(insert_sql, data)

In [11]:
insert_data_from_df(df_item, "item_info")
insert_data_from_df(df_outlet, "outlet_info")
insert_data_from_df(df_sales, "sales_info")

In [12]:
# Closing the cursor and the connection 
cursor.close()
conn.close()